# Exam 2
Author: Ian Malcolm

Special Thanks: 

### Imports

In [8]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.defaule = 'colab'

from itables import show

# This stops a few warning messages from showing
pd.options.mode.chained_assignment = None 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Load Data

In [13]:
# Load the data.
file_name = 'flights'
file_location = 'https://joannabieri.com/introdatascience/data/'+file_name+'.csv'
DF = pd.read_csv(file_location)
DF

year  month  day  dep_time  sched_dep_time  dep_delay  arr_time  \
0       2013      1    1     517.0             515        2.0     830.0   
1       2013      1    1     533.0             529        4.0     850.0   
2       2013      1    1     542.0             540        2.0     923.0   
3       2013      1    1     544.0             545       -1.0    1004.0   
4       2013      1    1     554.0             600       -6.0     812.0   
...      ...    ...  ...       ...             ...        ...       ...   
336771  2013      9   30       NaN            1455        NaN       NaN   
336772  2013      9   30       NaN            2200        NaN       NaN   
336773  2013      9   30       NaN            1210        NaN       NaN   
336774  2013      9   30       NaN            1159        NaN       NaN   
336775  2013      9   30       NaN             840        NaN       NaN   

        sched_arr_time  arr_delay carrier  flight tailnum origin dest  \
0                  819       11.0      UA    1545  N14228    EWR  IAH   
1                  830       20.0      UA    1714  N24211    LGA  IAH   
2                  850       33.0      AA    1141  N619AA    JFK  MIA   
3                 1022      -18.0      B6     725  N804JB    JFK  BQN   
4                  837      -25.0      DL     461  N668DN    LGA  ATL   
...                ...        ...     ...     ...     ...    ...  ...   
336771            1634        NaN      9E    3393     NaN    JFK  DCA   
336772            2312        NaN      9E    3525     NaN    LGA  SYR   
336773            1330        NaN      MQ    3461  N535MQ    LGA  BNA   
336774            1344        NaN      MQ    3572  N511MQ    LGA  CLE   
336775            1020        NaN      MQ    3531  N839MQ    LGA  RDU   

        air_time  distance  hour  minute            time_hour  
0          227.0    1400.0   5.0    15.0  2013-01-01 10:00:00  
1          227.0    1416.0   5.0    29.0  2013-01-01 10:00:00  
2          160.0    1089.0   5.0    40.0  2013-01-01 10:00:00  
3          183.0    1576.0   5.0    45.0  2013-01-01 10:00:00  
4          116.0     762.0   6.0     0.0  2013-01-01 11:00:00  
...          ...       ...   ...     ...                  ...  
336771       NaN     213.0  14.0    55.0  2013-09-30 18:00:00  
336772       NaN     198.0  22.0     0.0  2013-10-01 02:00:00  
336773       NaN     764.0  12.0    10.0  2013-09-30 16:00:00  
336774       NaN     419.0  11.0    59.0  2013-09-30 15:00:00  
336775       NaN     431.0   8.0    40.0  2013-09-30 12:00:00  

[336776 rows x 19 columns]

In [100]:
# Load the data.
file_name = 'airports'
file_location = 'https://joannabieri.com/introdatascience/data/'+file_name+'.csv'
ports_df = pd.read_csv(file_location)

file_name = 'airlines'
file_location = 'https://joannabieri.com/introdatascience/data/'+file_name
lines_df = pd.read_csv(file_location)

## Questions

### Preliminary

In [20]:
show(DF)

In [22]:
DF.dtypes

year                int64
month               int64
day                 int64
dep_time          float64
sched_dep_time      int64
dep_delay         float64
arr_time          float64
sched_arr_time      int64
arr_delay         float64
carrier            object
flight              int64
tailnum            object
origin             object
dest               object
air_time          float64
distance          float64
hour              float64
minute            float64
time_hour          object
dtype: object

Most of the variables are integers or floats, which should make the data easier to work with. From what I can tell the float fields seem to be essentially integers themselves, just with the option for NaN values. That is, except for distance, hour, and minute which seem like they should be pure integers due to not having NaN values.

Only carrier, flight, tailnum, origin, and dest are categorical variables. Maybe the time_hour field is also categorical, but I think that's a bit ambiguous as it can be treated as a timestamp.

### Question 1
What are the ten most common destinations for flights from NYC airports in 2013?

In [86]:
dest_counts = DF['dest'].value_counts()
dest_counts = pd.DataFrame(dest_counts)
dest_counts = pd.merge(ports_df[['name', 'faa']], dest_counts, right_on='dest', left_on='faa', how='right')
show(dest_counts)

top10dests = dest_counts.head(10)['name']
for dest in top10dests:
    print(dest, end=', ')

Chicago Ohare Intl, Hartsfield Jackson Atlanta Intl, Los Angeles Intl, General Edward Lawrence Logan Intl, Orlando Intl, Charlotte Douglas Intl, San Francisco Intl, Fort Lauderdale Hollywood Intl, Miami Intl, Ronald Reagan Washington Natl, 

The ten most common destinations for flights from NYC airports in 2013 were, in descending order, Chicago Ohare Intl, Hartsfield Jackson Atlanta Intl, Los Angeles Intl, General Edward Lawrence Logan Intl, Orlando Intl, Charlotte Douglas Intl, San Francisco Intl, Fort Lauderdale Hollywood Intl, Miami Intl, and Ronald Reagan Washington Natl.

### Question 2
Which airlines have the most flights departing from NYC airports in 2013?

In [104]:
origin_counts = DF['carrier'].value_counts()
origin_counts = pd.DataFrame(origin_counts)
origin_counts = pd.merge(lines_df[['name','carrier']], origin_counts, how='right',
                         on='carrier')

show(origin_counts)

The airlines with the most flights out of NYC in 2013 were United Airlines, JebBLue, and ExpressJet (in descending order).

### Question 3
Consider only flights that have non-missing arrival delay information and answer the questions below. Your answer should include the name of the carrier in addition to the carrier code and the values asked.

a. Which carrier had the highest mean arrival delay?

b. Which carrier had the lowest mean arrival delay?

In [162]:
arr_delays_df = DF[['arr_delay', 'carrier']][~DF['arr_delay'].isna()]

arr_delays_gb = arr_delays_df.groupby(by='carrier')
arr_delays_mean = arr_delays_gb['arr_delay'].mean()

arr_delays_mean = pd.DataFrame(arr_delays_mean)
arr_delays_mean = arr_delays_mean.rename(columns={'arr_delay': 'mean_arr_delay'})
arr_delays_mean = pd.merge(lines_df[['name','carrier']], arr_delays_mean, on='carrier', how='right')
arr_delays_mean = arr_delays_mean.sort_values(by='mean_arr_delay', ascending=False)
arr_delays_mean = arr_delays_mean.reset_index()

show(arr_delays_mean)
print(arr_delays_mean.head(1), arr_delays_mean.tail(1), sep='\n')

   index                    name carrier  mean_arr_delay
0      6  Frontier Airlines Inc.      F9       21.920705
    index                  name carrier  mean_arr_delay
15      2  Alaska Airlines Inc.      AS       -9.930889


a. The carrier with the longest mean arrival delay was Frontier.

b. The carrier with the shortest mean arrival delay was Alaska Airlines.

### Question 4
Consider breaking the day into four time intervals: 12:00am-6am, 6:01am-12pm, 12:01pm-6pm, 6:01pm-11:59pm, then answer the questions below.

a. Calculate the proportion of flights that are delayed at departure at each of these time intervals.

b. Comment on how the likelihood of being delayed change throughout the day? What is the probability of being delayed given each time period?

In [200]:
dep_delays_df = DF[['dep_delay', 'sched_dep_time']][~DF['dep_delay'].isna()]

dep_delays_df['delayed'] = (dep_delays_df['dep_delay'] > 0)

dep_delays_df['sched_int'] = pd.cut(dep_delays_df['sched_dep_time'],
                                    bins=[0,600,1200,1800,2400],
                                    labels=['00:00-06:00','06:01-12:00','12:01-18:00','18:00-23:59'],
                                    right=True)

dep_delays_gb = dep_delays_df.groupby(by='sched_int')

dep_delays_stats = pd.DataFrame({'total': dep_delays_gb['delayed'].count(),
                                 'delayed': dep_delays_gb['delayed'].sum()})

dep_delays_stats['proportion'] = dep_delays_stats['delayed']/dep_delays_stats['total']

show(dep_delays_stats)

a. Between 12am and 6am, about 21% of flights get delayed. About 26% of flights get delayed between 6am and 12pm, about 46% of flights get delayed between 12pm and 6pm, and about 52% of flights get delayed between 6pm and 12am.

b. The likelihood of departure delays goes up during over the course of the day. This is likely a result of flights getting backed up as the day passes. Because the high for total flights occurs in the middle of the day, the airlines have a chance to catch up in the evening, even as they hit their highest rate of delays during that time.

### Question 5
Find the flight with the longest air time.

a. How long is this flight?

b. What city did it fly to?

c. How many seats does the plane that flew this flight have?